<a href="https://colab.research.google.com/github/NanoNero1/IHT_AGD/blob/main/AGD_IHT_CLEAN_MNIST_SanityCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### AUTHOR: Dimitri Kachler

### This is a test cell for github

# Global Parameters

In [ ]:
#User-Dependent Variables
layerByLayer = False

# -------------- INACTIVE
#useNeptune = True

# Imports

In [ ]:
# Neural Networks
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required
import torch

# Arrays & Mathematics
import math
import numpy as np

#Plotting
import matplotlib.pyplot as plt
import pandas as pd

#System / IO
import abc
import itertools

#External Utilities
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [ ]:
# CUDA Check
print(torch.__version__)
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

2.3.0+cu121
Using cpu device


# Dataset Collection

SOURCE: https://colab.research.google.com/github/skorch-dev/skorch/blob/master/notebooks/MNIST.ipynb#scrollTo=mwpfASvc_v7J

SOURCE: https://github.com/pytorch/examples/blob/main/mnist/main.py

In [ ]:
""" Desc: The MNIST Dataset, sourced from PyTorch, this dataset is for classification of handwritten digits
  Size: 60,000 examples,
  Input: 28x28 pixels,
  Target: digit 0-9,
"""

# Data Collection and Normalizing so that it's suitable for input to the neural network
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)


# Data Loaders : These also allow us to test performance ad-hoc
train_loader = torch.utils.data.DataLoader(dataset1,batch_size=1000,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2,batch_size=1000,shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 54503292.75it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 12113569.38it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden




100%|██████████| 1648877/1648877 [00:00<00:00, 82858785.57it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5070675.74it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
print(type(dataset1))

<class 'torchvision.datasets.mnist.MNIST'>


# Building the Neural Network

SOURCE: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

SOURCE: https://colab.research.google.com/github/omarsar/pytorch_notebooks/blob/
[master/pytorch_quick_start.ipynb#scrollTo=3_0Vjq2RHlph](https://colab.research.google.com/github/omarsar/pytorch_notebooks/blob/master/pytorch_quick_start.ipynb#scrollTo=3_0Vjq2RHlph)


In [ ]:
### DEPRECATED
class basicNeuralNet(nn.Module):
  def __init__(self,inputDim,outputDim):
    super(basicNeuralNet, self).__init__()
    # Hyper-Parameters
    hiddenSize_1 = 200
    hiddenSize_2 = 50

    # Pytorch Layers
    self.flatten = nn.Flatten()
    self.hidden1 = nn.Linear(inputDim,hiddenSize_1)
    self.hidden2 = nn.Linear(hiddenSize_1,hiddenSize_2)
    self.prob = nn.Linear(hiddenSize_2,outputDim)

    # Activation Functions
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax()

  # Forward Pass: outputs a vector [x] of softmax probabilities
  def forward(self,x):
    flat = self.flatten(x)
    hidden_1 = self.relu(self.hidden1(flat))
    hidden_2 = self.relu(self.hidden2(hidden_1))
    prob = self.prob(hidden_2)
    out =  F.log_softmax(prob, dim=1)
    return out

""" Desc: A basic Convolutional Neural Network : these are particularly suitable for image-based problems
"""
class convNet(nn.Module):
    def __init__(self):
        super(convNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 1)
        self.fc1 = nn.Linear(4608, 128)
        self.fc2 = nn.Linear(128, 10)

    # Forward Pass: outputs a vector [x] of softmax probabilities
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        # Softmax so that we output probabilities (i.e. adds up to 1)
        output = F.log_softmax(x, dim=1)
        return output

# Training & Testing Functions

SOURCE: https://github.com/pytorch/examples/tree/main/mnist

In [ ]:
""" Desc: function for one training epoch. At each step, we query the optimizer and log the training loss to Neptune.
  Output: training loss, testing loss, [! we do not capture testing accuracy here]
"""

def train(args, model, device, train_loader, optimizer, epoch,trialNumber=None):
    model.train()

    train_epoch_losses = []
    test_epoch_losses = []

    # In case we get NaN, setting this to true should detect this
    # BUG: for some reason this fails to capture NaNs
    torch.autograd.set_detect_anomaly(True)

    # Making sure we know what scheme is implemented
    print(optimizer.methodName)

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # Generating Predictions and Calculating Loss
        output = model(data)
        loss = F.nll_loss(output, target)

        # Logs to Neptune AI
        run[f"trials/{optimizer.trialNumber}/training/batch/loss/{optimizer.methodName}/"].append(loss)

        ### DEPRECATED
        # Saving Loss Data
        train_epoch_losses.append(loss.cpu().detach().numpy())
        if batch_idx % 5 == 0:
          #test_epoch_losses.append(getTestLoss(model, device, test_loader))

          # For now we are not collecting the test loss -- we will do this later on
          test_epoch_losses.append(0)

        # Backwards Propogation and Calling the Optimizer
        loss.backward()

        #NOTE: Is there going to be a naming conflict with 'params'?
        def getNewGrad(params):
          model.params = params

          newOutput = model(data)
          loss = F.nll_loss(newOutput, target)
          loss.backward()

        # New Optimization Step
        optimizer.step(getNewGrad=getNewGrad)
        #print(f"learning rate: {optimizer.lr}")

        ### DEPRECATED - On every 10 iterations, we can print out some information
        if batch_idx % 10 == 0 :
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    return train_epoch_losses,test_epoch_losses


def getTestLoss(model, device, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
      data, target = next(iter(test_loader))
      data, target = data.to(device), target.to(device)
      output = model(data)

      # Negative Log-Likelihood Loss
      test_loss += F.nll_loss(output, target, reduction='sum').item()
    # The total loss is divided by the batch size to get the average
    test_loss /= test_loader.batch_size
    return test_loss


def getTestAccuracy(model, device, test_loader):
    model.eval()
    correct = 0
    # The testing accuracy is taken over the entire dataset
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # If the index of the maximum probability matches the target digit, we add it to the corrrect counter variable
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / len(test_loader.dataset)

    print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * accuracy))
    return accuracy

# Custom Optimizer

SOURCE: https://www.geeksforgeeks.org/custom-optimizers-in-pytorch/

SOURCE: https://github.com/azotlichid/adaptive-opt

# Test Optimizer Module?

In [ ]:
""" Desc: The base optimizer class inherits from the PyTorch Optimizer class. The idea of this design is that we can make brand new
optimizers without having to re-write a lot of the boilerplate code. It also makes it really easy to combine different methods,
e.g. Iterative Hard Thresholding with Accelerated Gradient Descent is a combination of IHT-SGD and vanilla-AGD.
"""

class myOptimizer(Optimizer):
  def __init__(self,params,lr=1.0,sparse=False,demoValue=100):

    # A dummy value to see if inheritance works properly
    self.demoValue = demoValue

    self.lr = lr
    defaults = dict(lr=lr,sparse=sparse)
    super(myOptimizer, self).__init__(params,defaults) # For pytorch's Optimizer class

    self.iteration = 0
    self.trialNumber = None

    self.methodName="base_optimizer"

    ### These methods should be overridden after inheritance
    """ Desc: the main function that the optimizer gets called on every iteration """
    @abc.abstractmethod
    def step(self,getNewGrad):
      return None

    """ Desc: what optimization scheme it uses to update weights, e.g. Accelerated Gradient Descent """
    @abc.abstractmethod
    def updateWeights(self):
      pass

    ### Methods that every optimizer needs to use

    def toNeptuneLoss():
      pass

    def logging():
      pass

# ----------------------------------------------------Vanilla SGD

class vanillaSGD(myOptimizer):
  def __init__(self,params,lr=1.0):
    super(vanillaSGD, self).__init__(params, lr=lr)
    self.methodName = "vanilla_SGD"

  def step(self,getNewGrad):
    self.updateWeights()
    self.iteration +=1
    return None

  # Regular Gradient Descent
  def updateWeights(self):
    for group in self.param_groups:
      for p in group['params']:
        #Everyone gets updated at the same time because it is easier
        p.data -= p.grad*self.lr

# ---------------------------------------------------- IHT SGD

class ihtSGD(vanillaSGD):
  def __init__(self, params, lr=1.0,getNewGrad=None,sparsity=0.9,sparsifyInterval=10):
    super(ihtSGD, self).__init__(params, lr=lr)
    self.sparsity = sparsity
    self.sparsifyInterval = sparsifyInterval

    # For freezing weights
    self.frozen = [torch.ones_like((p.to(device))) for p in self.param_groups[0]['params']]

    self.methodName = "iht_SGD"

  def step(self,getNewGrad):
    self.updateWeights()
    self.sparsify()
    self.iteration +=1
    return None

  """ Desc: sparsifies the current weights according to a truncation operator - implementations may vary """
  def sparsify(self):
    concatWeights = torch.zeros((1)).to(device)
    for group in self.param_groups:
      for p in group['params']:
        flatWeights = torch.flatten(p.data)
        concatWeights = torch.cat((concatWeights,flatWeights),0)

    # Converting the sparsity factor into an integer of respective size
    topK = int(len(concatWeights)*(1-self.sparsity))

    # All the top-k values are sorted in order, we take the last one as the cutoff
    vals, bestI = torch.topk(torch.abs(concatWeights),topK,dim=0)
    cutoff = vals[-1]

    # Zero-ing out values below the cutoff
    for group in self.param_groups:
      for p in group['params']:
        p.data[abs(p.data) <= cutoff] = 0.0

  """ Desc: freezes the previously truncated weights - should be done before sparsification and during the freezing phase"""
  ### INACTIVE, but IMPLEMENTED
  def freeze(self):
    for layerIdx,layer in enumerate(self.frozen):
      self.zt[layerIdx] = self.frozen[layerIdx] * self.zt[layerIdx]
      self.xt[layerIdx] = self.frozen[layerIdx] * self.xt[layerIdx]
      #self.param_groups[0]['params'][layerIdx] = self.frozen[layerIdx] * self.param_groups[0]['params'][layerIdx]
      self.param_groups[0]['params'][layerIdx].data *= self.frozen[layerIdx]
    else:
      print('regular step')

  # TODO: move the code from the old AGD class here
  #def updateFrozen():


# ---------------------------------------------------- Vanilla AGD

class vanillaAGD(vanillaSGD):
  def __init__(self,params,kappa=5.0,beta=50.0):
    super(vanillaAGD, self).__init__(params)
    self.methodName = "vanilla_AGD"

    # Objective Function Property Variables
    # NOTE: good values are found with Grid Search
    self.alpha = beta / kappa
    self.beta = beta
    self.kappa = kappa
    self.sqKappa = pow(self.kappa,0.5)

    # Initializing the internal storage for the iterates
    self.zt = [torch.zeros_like((p.to(device))) for p in self.param_groups[0]['params']]
    self.xt = [p for p in self.param_groups[0]['params']]

    # Compression, Decompression and Freezing Variables
    self.phaseLength = 100
    self.compressionRatio = 0.5
    self.freezingRatio = 0.5

    self.methodName = "vanilla_AGD"

  def step(self,getNewGrad):
    self.updateWeights(getNewGrad)
    self.iteration += 1


  """ Desc: a function that decides wether to process in the compressed or decompressed phase, or the freezing phase"""
  # INACTIVE, NOT TESTED
  def compressOrDecompress(self,getNewGrad):
    howFarAlong = self.iteration % self.phaseLength

    if howFarAlong >= self.phaseLength * self.compressionRatio:
      self.updateWeights(getNewGrad)
    elif howFarAlong >= self.phaseLength * self.compressionRatio * self.freezingRatio:
      # INACTIVE
      self.updateWeights(getNewGrad)
      self.sparsify()
      # NOTE: is the order correct?
      self.freeze()
    else:
      self.updateWeights(getNewGrad)
      self.sparsify()

  def updateWeights(self,getNewGrad):

    ############################## DO NOT FORGET ###################################
    # For efficiency purposes I will need to store old gradient z_{t-1} because we re-use it in the current iteration
    # right now it takes 1.5 as much computation as we really need
    ################################################################################

    # We personally query a new gradient from a custom iterate z_{t-1}
    getNewGrad(self.zt)

    # Update z_t the according to the AGD equation in the note
    for layerIdx,layer in enumerate(self.zt):
      self.zt[layerIdx] = (self.sqKappa / (self.sqKappa + 1.0) ) * (self.zt[layerIdx] - self.param_groups[0]['params'][layerIdx].grad / self.beta) + (1.0 / (self.sqKappa + 1.0)) * self.xt[layerIdx]

    # This is actually \grad z_t
    getNewGrad(self.zt)

    # Final Step of Accelerated Gradient Descent
    for group in self.param_groups:
      for pInd,p in enumerate(group['params']):
        #Everyone gets updated at once because it is easier to implement
        p.data -= p.grad*(1.0 / pow(self.alpha*self.beta,0.5))

# ---------------------------------------------------- IHT AGD

class ihtAGD(vanillaAGD,ihtSGD):
  def __init__(self,params,sparsity=0.9,kappa=5.0,beta=50.0):
    super(ihtAGD, self).__init__(params)
    self.methodName = "iht_AGD"
    self.alpha = beta / kappa
    self.beta = beta
    self.kappa = kappa

    # BUG: Because of multiple inheritance I need to add this statement below,
    # but I know how to fix this
    self.sparsity=sparsity

  def step(self,getNewGrad):
    self.updateWeights(getNewGrad)
    self.sparsify()
    self.iteration += 1


# Experiment Trial Definition

In [ ]:
# NOTE: we want to remove this and replace it with a better system,
# nonetheless, this is still useful for now due to a small issue in my setup code
def chooseOptimizer(setup,model,trialNumber):
  match setup["scheme"]:
    case "vanillaSGD":
         optimizer = vanillaSGD(model.parameters(),lr=setup["lr"])
    case "ihtSGD":
         optimizer = ihtSGD(model.parameters(),sparsity=setup["sparsity"],lr=setup["lr"])
    case "vanillaAGD":
         optimizer = vanillaAGD(model.parameters(),kappa=setup["kappa"],beta=setup["beta"])
    case "ihtAGD":
         optimizer = ihtAGD(model.parameters(),sparsity=setup["sparsity"],kappa=setup["kappa"],beta=setup["beta"])
    case "untouchedIhtAGD":
        optimizer = untouchedIhtAGD(model.parameters(),sparsity=setup["sparsity"],kappa=setup["kappa"],beta=setup["beta"])
    case _:
        pass
        #action-default
  optimizer.trialNumber = trialNumber
  return optimizer

def runOneExperiment(setup=None,trialNumber=None):
  model = convNet().to(device)

  optimizer = chooseOptimizer(setup,model,trialNumber)
  #optimizer = eval(setup["scheme"])(setup,model,trialNumber)

  training_losses = []
  testing_losses = []
  testing_accuracies = []

  # This implementation uses a Learning Rate Scheduler
  # TO-DO: test if it actually changes the learning rate, I suspect it isn't working
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)
  for epoch in range(1, setup["epochs"] + 1):

    print(optimizer.methodName)

    # Call to run one epoch of training
    train_epoch_loss,test_epoch_loss = train([],model, device, train_loader, optimizer, epoch,trialNumber)

    training_losses.extend(train_epoch_loss)
    testing_losses.extend(test_epoch_loss)


    testAccuracy = getTestAccuracy(model, device, test_loader)

    # Logs to Neptune AI
    run[f"trials/{optimizer.trialNumber}/training/batch/accuracy/{optimizer.methodName}/"].append(testAccuracy)

    testing_accuracies.append(testAccuracy)

    scheduler.step()

  print(testing_accuracies)
  return model,training_losses,testing_losses,testing_accuracies

In [ ]:
def runMainExperiment(setups,epochs=5):
  defaults = {"epochs":epochs}

  # Combines the default setup with some added parameters
  setups = [defaults | setup for setup in setups]

  print(setups)

  all_training_losses = [[] for i in range((len(setups)))]
  all_testing_losses = [[] for i in range((len(setups)))]
  all_models = [[] for i in range((len(setups)))]
  all_accuracies = [[] for i in range((len(setups)))]
  for idx in range(len(setups)):
    all_models[idx],all_training_losses[idx],all_testing_losses[idx],all_accuracies[idx] = runOneExperiment(setups[idx],idx)

  return all_models,all_training_losses,all_testing_losses,all_accuracies

# Setups

In [ ]:
setup_ihtAGD = {
    "scheme":"ihtAGD" ,
    "lr":0.1,
    "sparsity":0.90,
    "kappa":3.0,
    "beta":50.0}

setup_vanillaAGD = {
    "scheme":"vanillaAGD",
    "lr":0.1,
    "sparsity":0.9,
    "kappa":3.0,
    "beta":50.0,}
# NOTE: you can add in more variables than necessary, e.g. AGD doesn't use the learning rate 'lr'
# NOTE 2: do we want it to ignore extra parameters? so far ignoring has been useful in the implementation

setup_ihtSGD = {
    "scheme":"ihtSGD" ,
    "lr":0.1,
    "sparsity":0.90,}

setup_vanillaSGD = {
    "scheme":"vanillaSGD",
    "lr":0.1,
    "sparsity":0.9,}

#IMPORTANT! re-run this in the same cell, otherwise if you change the setups it won't work for some reason (maybe because the array casts the variables)
setups = [setup_ihtAGD,setup_vanillaAGD,setup_ihtSGD,setup_vanillaSGD]

# Neptune.ai

In [ ]:
# NOTE: you can still run this and it should still work and send the data to my Neptune.ai project,
# unfortunately you won't be able to see the graph without my account
# TO-DO: add other people to the neptune project, or export back to colab in some way with a CSV output.
%pip install -U neptune
import neptune
from getpass import getpass

project="dimitri-kachler-workspace/sanity-MNIST"
api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlNWQxNDllOS04OGY1LTRjM2EtYTczZi0xNWI0NTRmZTA1OTEifQ=="

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.2 MB/s eta 0:00:00
  Created wheel for bravado-core: filename=bravado_core-6.1.1-py2.py3-none-any.whl size=67672 sha256=5a49c8cb3a311f4af6f3a9ed6cda7c39c1348cb0850d08c7ae254388cc9b494e
  Stored in directory: /root/.cache/pip/wheels/42/35/4a/44ec

In [ ]:
### BUG: For whatever reason I can't use Neptune in a function like this, it doesn't log anything
def runWithNeptune(setups):
  run = neptune.init_run(api_token=api_token, project=project)
  all_models,all_training_losses,all_testing_losses,all_accuracies = runMainExperiment(setups)
  run.stop()
  return all_models,all_training_losses,all_testing_losses,all_accuracies

# Running the Experiment

In [ ]:
""" MAIN CELL """
run = neptune.init_run(api_token=api_token, project=project)
all_models,all_training_losses,all_testing_losses,all_accuracies = runMainExperiment(setups)
run.stop()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/dimitri-kachler-workspace/sanity-MNIST/e/SAN-129
[{'epochs': 5, 'scheme': 'ihtAGD', 'lr': 0.1, 'sparsity': 0.9, 'kappa': 3.0, 'beta': 50.0}, {'epochs': 5, 'scheme': 'vanillaAGD', 'lr': 0.1, 'sparsity': 0.9, 'kappa': 3.0, 'beta': 50.0}, {'epochs': 5, 'scheme': 'ihtSGD', 'lr': 0.1, 'sparsity': 0.9}, {'epochs': 5, 'scheme': 'vanillaSGD', 'lr': 0.1, 'sparsity': 0.9}]
iht_AGD
iht_AGD
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305031
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.542139
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.913042
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.736972
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.673354
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.483504

Accuracy: 8592/10000 (86%)

iht_AGD
iht_AGD
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.452763
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.402066
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.438960
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.355656
Tr

KeyboardInterrupt: 

# END OF THE BASELINE FRAMEWORK, NEXT SECTION DEDICATED TO EXTENSIONS


# Bias Left Untouched

In [ ]:
class untouchedIhtAGD(ihtAGD):
  def __init__(self,params,sparsity=0.9,kappa=5.0,beta=50.0):
    super().__init__(params)
    self.methodName = "untouched_iht_AGD"
    self.alpha = beta / kappa
    self.beta = beta
    self.kappa = kappa

  def sparsify(self):
    # TO-DO: remember to remove this zero, it is inconsequential, but still remove it in good practice
    concatWeights = torch.zeros((1)).to(device)
    for group in self.param_groups:
      for p in group['params']:

        #Skip Bias Layers
        if len(p.data.shape) > 1:
          continue

        flatWeights = torch.flatten(p.data)
        concatWeights = torch.cat((concatWeights,flatWeights),0)

    topK = int(len(concatWeights)*(1-self.sparsity))
    vals, bestI = torch.topk(torch.abs(concatWeights),topK,dim=0)
    cutoff = vals[-1]
    for group in self.param_groups:
      for p in group['params']:

        #Skip Bias Layers
        if len(p.data.shape) > 1:
          continue

        p.data[abs(p.data) <= cutoff] = 0.0


In [ ]:
setup_untouched_ihtAGD = {
    "scheme":"untouchedIhtAGD",
    "lr":0.1,
    "sparsity":0.90,
    "kappa":10.0,
    "beta":100.0}
setups = [setup_untouched_ihtAGD, setup_ihtAGD]

run = neptune.init_run(api_token=api_token, project=project)
all_models,all_training_losses,all_testing_losses,all_accuracies = runMainExperiment(setups)
run.stop()

# Grid Search

In [ ]:
from os import setgroups

def gridSearch(default,variables,values,metric,epochs=1):
  """ Desc: searches in a grid for the best combination of values of arbitrary dimension,
        we can check for more than 2 variables at a time, but this can be very costly

  default [dictionary]: a dictionary for all the default settings, this is also how one can set the type of algorithm
  variables [array[string]]: the settings to change
  values [2Darray]: what values to take on
  metric [string]: what metric to use for the best value
  """

  # We will not know how to traverse this list easily however
  # TO-DO: find a way to organize, or traverse this list
  setups = []

  # This list has every possible combination of the settings
  valuePermutations = list(itertools.product(*values))

  for permutation in valuePermutations:
    newSetup = default.copy()
    for idx,val in enumerate(permutation):

      # Adjusts the settings one-by-one
      newSetup[variables[idx]] = val

    setups.append(newSetup)

  print(setups)


  all_models,all_training_losses,all_testing_losses,all_accuracies = runMainExperiment(setups,epochs=epochs)

  # NEXT: Interchange with a different metric
  # TO-DO: try "highest loss" over entire dataset using model

  # Right now we use the accuracy in after the last epoch
  # BUG: is the last epoch at 0 or -1 I need to check
  min_accuracies = [accuracies[-1] for accuracies in all_accuracies]
  bestSetupIndex = min_accuracies.index(min(min_accuracies))



  return setups[bestSetupIndex]

In [ ]:
default = {
    "scheme":"vanillaAGD",
    "lr":0.1,
    "sparsity":0.90,
    "kappa":15.0,
    "beta":10000.0}
# We set a big value to see if we overwrite it in the Grid Search

gridSearch(default,["kappa","beta"],[[2.0,10.0,100.0],[10.0,100.0,300.0]],"loss",5)

In [ ]:
#This works! It recognizes it as a class name
type(eval("ihtAGD"))

# END OF THE CODE

# **Appendix**

# Plotting

In [ ]:
def plotAllAccuracies(all_accuracies):
  # TO-DO: make dynamic instead of set labels
  sparses = [0.5,0.9,0.95]
  for idx,accuracies in enumerate(all_accuracies):
    xes = [i for i in range(len(accuracies))]
    plt.plot(xes,np.array(accuracies)*100,label=f"Sparsity: {sparses[idx]}")
  plt.ylim([80,100])
  #plt.yscale("log")
  plt.title("MNIST - Accuracies")
  plt.xlabel("Training Iterations")
  plt.ylabel("Accuracy (%)")
  plt.legend(loc="upper right")
  plt.show()
plt.figure(figsize=(20,14))
plotAllAccuracies(all_accuracies)

In [ ]:
def plotAllLosses(all_training,all_testing):
  #print(list_losses)
  sparses = [0.5,0.9,0.95]
  for idx,train_loss in enumerate(all_training):
    xes = [i for i in range(len(train_loss))]
    #plt.plot(xes,train_loss,label='Training Loss',color='turquoise',linestyle='dashed')
    plt.plot(xes,train_loss,label=f"Train: {sparses[idx]}")
  for idx,test_loss in enumerate(all_testing):
    print(test_loss)
    xes = [i for i in range(len(test_loss))]
    #plt.plot(xes,test_loss,label=f"Test: {sparses[idx]}",linestyle='dashed')
  #plt.ylim([0,1.0])
  #plt.yscale("log")
  plt.title("MNIST - FIRST TRIAL")
  plt.xlabel("Training Iterations")
  plt.ylabel("Negative Log-Likelihood Loss")
  plt.legend(loc="upper right")
  plt.show()


plt.figure(figsize=(20,14))
plotAllLosses(all_training_losses,all_testing_losses)

# Saving and Loading Model

SOURCE: https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [ ]:
def saveModel(model,pathdir):
  torch.save(model.state_dict(), pathdir)

def loadModel(pathdir,modeltype):
  match modeltype:
    case "basicNeuralNet": model = basicNeuralNet(784,10).to(device)
    case "convNet": model = convNet().to(device)

  model.load_state_dict(torch.load(pathdir))
  model.eval()
  return model

In [ ]:
saveModel(all_models[0],"testModel")

In [ ]:
tryModel = loadModel("testModel","convNet")

# Notes

Sparsify Interval
Base case
Fine-Tuning Phase (Freeze weights) , < Sparsify interval
Real-time visualization - add trainin loss per batch and test loss, and test accuracy
Weights and Biases


AC/DC proof 8.1.4,

Make proof on board work for large numbers, i.e.! T:(S* times Kappa^2 * some constant factor)
Want the damage to be 1 + epsilon

make sure you can collect useful information - e.g. things like sparsity




# Seaborn Integration

In [ ]:
df2 = pd.DataFrame()
df2["idp"] = np.array([1,2,3,4,5])
df2["dep"][0] = np.array([3,2,6,4,8])
df2["dep"][1] = np.array([2,2,3,1,3])

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")

sns.lineplot(x="idp", y="dep",data=df2)

In [ ]:
def plotTrainingLosses(data):
  sns.lineplot(y="training_losses",data=data)
  return None
def plotTestingLosses(data):
  return None
def plotAccuracies(data):
  return None

# Empirically Testing the Model

In [ ]:
def testModel(model):
  randomExampleInt = np.random.randint(1000)
  exampleX = dataset2.data[randomExampleInt].reshape(28, 28)
  plt.imshow(exampleX)
  print(exampleX.shape)
  exampleX = torch.reshape(exampleX, (1, 1,28,28))
  predicted = model(torch.tensor(exampleX,dtype=torch.float32).to(device))
  print(torch.argmax(predicted))

testModel(tryModel)